In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import nnfs
from nnfs.datasets import sine_data, spiral_data
import random
import requests
from NNS import NeuralNetwork as NN #import neural net code from github to reduce copy/pasting
from NNS import load_MNIST_Data
from tqdm import tqdm
from sklearn.neighbors import KNeighborsClassifier
import os

# Saving and Loading Models and Their Paraneters

## Retrieving Parameters

In [ ]:
# Add this to Dense layer
def get_parameters(self):
    return self.weights, self.biases

# Add this to Model class
# Retrieves and returns parameters of trainable layers
def get_parameters (self):
    
    # Create a list for parameters
    parameters = []
    
    # Iterable trainable layers and get their parameters
    for layer in self.trainable_layers:
        parameters.append(layer.get_parameters())
        
    # Return a list
    return parameters

## Setting Parameters

In [ ]:
# Add this to Dense Layer

# Set weights and biases in a layer instance
def set_parameters(self, weights, biases):
    self.weights = weights
    self.biases = biases
    
    
# Add this to the Model Class
  
# Updates the model with new parameters
def set_parameters(self, parameters):
    # Iterate over the parameters and layers
    # and update each layers with each set of the parameters
    for parameter_set, layer in zip(parameters, self.trainable_layers):
        layer.set_parameters( * parameter_set)
        
# Update Model Finalize function to account for not needing an optimizer
def finalize(self):
    """"""
    # Update loss object with trainable layers
    if self.loss is not None:
        self.loss.remember_trainable_layers(self.trainable_layers)
        
# Update Model Set function to account for taking in paramters
# Set loss, optimizer and accuracy
def set(self , *, loss = None, optimizer = None, accuracy = None):
    
    if loss is not None:
        self.loss = loss
    
    if optimizer is not None :
        self.optimizer = optimizer
    
    if accuracy is not None :
        self.accuracy = accuracy

## Saving Parameters

In [ ]:
import pickle # For serializing  nidek ivject

# Add save parameters function to model
# Saves the parameters to a file
def save_parameters(self, path):
    
    # Open a file in the binary-write mode and save parameters to it
    with open (path, 'wb') as f:
        pickle.dump(self.get_parameters(), f)

## Loading Parameters

In [ ]:
# Loads the weights and updates a model instance with them
def load_parameters(self, path):
    # Open file in the binary-read mode,
    # load weights and update trainable layers
    with open (path, 'rb' ) as f:
        self.set_parameters(pickle.load(f))

## Saving the Model

In [ ]:
# Saves the model
import copy

def save(self, path):
    
    # Make a deep copy of current model instance
    model = copy.deepcopy(self)
    
    # Reset accumulated values in loss and accuracy objects
    model.loss.new_pass()
    model.accuracy.new_pass()
    
    # Remove data from the input layer and gradients from the loss object
    model.input_layer.__dict__.pop('output' , None)
    model.loss.__dict__.pop('dinputs', None)
    
    # For each layer remove inputs, output and dinputs properties
    for layer in model.layers:
        for property in [ 'inputs', 'output', 'dinputs', 'dweights', 'dbiases']:
            layer.__dict__.pop( property , None )
    
    # Open a file in the binary-write mode and save the model
    with open (path, 'wb' ) as f:
        pickle.dump(model, f)

## Loading the model

In [ ]:
# Loads and returns a model
@staticmethod #create a model object without first needing to instantiate a model object
def load(path):
    
    # Open file in the binary-read mode, load a model
    with open (path, 'rb' ) as f:
        model = pickle.load(f)
        
    # Return a model
    return model

In [ ]:
# Load the data
URL = 'https://nnfs.io/datasets/fashion_mnist_images.zip'
FILE = 'fashion_mnist_images.zip'
FOLDER = 'fashion_mnist_images'

#load_MNIST_Data.download_mnist_dataset(URL = URL, FILE = FILE, FOLDER = FOLDER)
X, y, X_test, y_test = load_MNIST_Data.create_data_mnist('fashion_mnist_images' )

# Shuffle the training dataset
keys = np.array( range (X.shape[ 0 ]))
np.random.shuffle(keys)
X = X[keys]
y = y[keys]

# Scale and reshape samples
X = (X.reshape(X.shape[ 0 ], - 1 ).astype(np.float32) - 127.5 ) / 127.5
X_test = (X_test.reshape(X_test.shape[ 0 ], - 1 ).astype(np.float32) - 127.5 ) / 127.5

In [2]:
#load_MNIST_Data.download_mnist_dataset(URL = URL, FILE = FILE, FOLDER = FOLDER)
X, y, X_test, y_test = load_MNIST_Data.create_data_mnist( 'fashion_mnist_images' )

# Shuffle the training dataset
keys = np.array(range(X.shape[0]))
np.random.shuffle(keys)
X = X[keys]
y = y[keys]

# Scale and reshape samples
X = (X.reshape(X.shape[ 0 ], - 1 ).astype(np.float32) - 127.5 ) / 127.5
X_test = (X_test.reshape(X_test.shape[ 0 ], - 1 ).astype(np.float32) - 127.5 ) / 127.5
# Instantiate the model
model = NN.Model()
# Add layers
model.add(NN.Layer_Dense(X.shape[ 1 ], 128 ))
model.add(NN.Activation_ReLU())
model.add(NN.Layer_Dense( 128 , 128 ))
model.add(NN.Activation_ReLU())
model.add(NN.Layer_Dense( 128 , 10 ))
model.add(NN.Activation_Softmax())
model.set(
loss = NN.Loss_CategoricalCrossentropy(),
    optimizer = NN.Optimizer_Adam( decay = 1e-3 ),
    accuracy = NN.Accuracy_Categorical()
)

# Finalize the model
model.finalize()
# Train the model
model.train(X, y, validation_data = (X_test, y_test),
            epochs = 10 , batch_size = 128 , print_every = 100 )

print("Evaluate testing data: ")
model.evaluate(X_test, y_test)
print("Evaluate training data: ")
model.evaluate(X, y)

model.save( 'fashion_mnist.model' )

epoch: 1
step: 0 , acc: 0.109, loss: 2.303 (data_loss: 2.303 , reg_loss: 0.000), lr: 0.001
step: 100 , acc: 0.648, loss: 0.781 (data_loss: 0.781 , reg_loss: 0.000), lr: 0.0009090909090909091
step: 200 , acc: 0.812, loss: 0.484 (data_loss: 0.484 , reg_loss: 0.000), lr: 0.0008333333333333334
step: 300 , acc: 0.820, loss: 0.486 (data_loss: 0.486 , reg_loss: 0.000), lr: 0.0007692307692307692
step: 400 , acc: 0.820, loss: 0.539 (data_loss: 0.539 , reg_loss: 0.000), lr: 0.0007142857142857143
step: 468 , acc: 0.812, loss: 0.457 (data_loss: 0.457 , reg_loss: 0.000), lr: 0.000681198910081744
training, acc: 0.757, loss: 0.659 (data_loss: 0.659, reg_loss: 0.000 ), lr: 0.000681198910081744
validation, acc: 0.824,loss: 0.487
epoch: 2
step: 0 , acc: 0.859, loss: 0.453 (data_loss: 0.453 , reg_loss: 0.000), lr: 0.0006807351940095304
step: 100 , acc: 0.781, loss: 0.523 (data_loss: 0.523 , reg_loss: 0.000), lr: 0.0006373486297004461
step: 200 , acc: 0.883, loss: 0.335 (data_loss: 0.335 , reg_loss: 0.000

In [3]:
# Load the model
model = NN.Model.load( 'fashion_mnist.model' )
# Evaluate the model
model.evaluate(X_test, y_test)

validation, acc: 0.872,loss: 0.359
